In [ ]:
import numpy as np

In [ ]:
# path = '/home/rajeev/rrd/LfF/dataset/ColoredMNIST-Skewed0.01-Severity4/valid/attrs.npy'
path = '/home/rajeev/rrd/LfF/dataset/ColoredMNIST-Skewed0.01-Severity4/train/attrs.npy'

# path = '/home/rajeev/rrd/LfF/dataset/CorruptedCIFAR10-Type1-Skewed0.01-Severity4/train/attrs.npy'
# path = '/home/rajeev/rrd/LfF/dataset/CorruptedCIFAR10-Type1-Skewed0.01-Severity4/valid/attrs.npy'

attrs = np.load(path)
print(attrs.shape)

In [ ]:
# print(attrs[:, 0])
# print(attrs[:, 1])
print(attrs)


In [ ]:
# count attrs[:, 0] == attrs[:, 1]
# print(np.sum(attrs[:, 0] == attrs[:, 1]))
t=(np.sum(attrs[:, 0] == attrs[:, 1])/attrs.shape[0])
t


In [ ]:
def calculate_cosine_similarity_loss(model_b, model_d, layer_names):
    cosine_loss = 0.0
    for name1, param1 in model_d.named_parameters():
        print(f'layer name: {name1}')
        # print(f'param1: {param1}')
        # print(f'param size: {param1.size()}')
        for layer_name in layer_names:
            if layer_name in name1:
                name2 = name1.replace(layer_name, layer_name.replace('features', 'classifier'))
                # print(f'name2: {name2}')
                param2 = model_d.state_dict()[name2]
                # print(f'param2: {model_d.state_dict()[name2]}')
                if param1 is not None and param2 is not None and 'weight' in name1:
                    weight1 = param1.view(param1.size(0), -1)
                    # print(f'weight1: {weight1}')
                    weight2 = param2.view(param2.size(0), -1)
                    # print(f'weight2: {weight2}')
                    cosine_similarity = F.cosine_similarity(weight1, weight2)
                    # print(f'cosine_similarity: {cosine_similarity}')
                    cosine_loss += 1 - cosine_similarity
                    # print(f'cosine_loss: {cosine_loss}')

                    # plot the cosine_loss using matplotlib
                    # plt.plot(cosine_loss)
    return cosine_loss

#

# Checking if Models trained have orthogonal weights in layers 

In [2]:
import torch
import torch.nn.functional as F

model_d = torch.load('/home/rajeev/rrd/LFF/LOGS/log_new_10June/colored_mnist/result/ColoredMNIST-Skewed0.01-Severity4/model_d_cosine__neg__0.01__24.th')
state_dict1 = model_d['state_dict']
model_b= torch.load('/home/rajeev/rrd/LFF/LOGS/log_new_10June/colored_mnist/result/ColoredMNIST-Skewed0.01-Severity4/model_b_cosine__neg__0.01__24.th')
state_dict2 = model_b['state_dict']

F.cosine_similarity(state_dict1['feature.2.weight'],state_dict2['feature.2.weight']).mean()

# 


tensor(0.9984, device='cuda:0')

# Cosine Functions

### No Gradinet Update

In [ ]:
def calculate_cosine_similarity_loss(model_b, model_d, layer_names):
    cosine_loss = 0.0
    for layer_name in layer_names:
        param1 = model_b.state_dict()[layer_name]
        param2 = model_d.state_dict()[layer_name]
        param1.requires_grad = True  # Set requires_grad to True
        param2.requires_grad = True  # Set requires_grad to True
        weight1 = param1.view(param1.size(0), -1)
        weight2 = param2.view(param2.size(0), -1)
        cosine_similarity = F.cosine_similarity(weight1, weight2)
        cosine_loss += 1 - cosine_similarity
        print(f'cosine_loss: {cosine_loss.mean()}')
    return cosine_loss


### Type: 1
#### lambda_ortho = 0.01
# 
``` 
        layer_names = ['feature.2.weight', 'feature.4.weight']
        ortho_loss = calculate_cosine_similarity_loss(model_d, model_b, layer_names)
        loss = loss_b_update.mean() + loss_d_update.mean() - (lambda_ortho / 2) * ortho_loss
``` 
- Results: 74.87%  

In [ ]:

def calculate_cosine_similarity_loss(model_b, model_d, layer_names):
    cosine_loss = 0.0
    for name1, param1 in model_b.named_parameters():
        if 'weight' in name1 and layer_names[0] in name1:
            param2 = model_d.state_dict().get(name1)
            if param1 is not None and param2 is not None:
                weight1 = param1.view(param1.size(0), -1)
                weight2 = param2.view(param2.size(0), -1)
                cosine_similarity = F.cosine_similarity(weight1, weight2)
                cosine_loss += 1 - cosine_similarity
    return cosine_loss

#

### Type: 2
#### lambda_ortho = 0.01
# 
``` 
        layer_names = [2,4]
        ortho_loss = calculate_cosine_similarity_loss(model_d, model_b, layer_names)
        loss = loss_b_update.mean() + loss_d_update.mean() - (lambda_ortho / 2) * ortho_loss
``` 
- Results: 72.80%  

In [ ]:
def calculate_cosine_similarity_loss(model_b, model_d, layer_names):
    cosine_loss=0.0
    for layer in layer_names:
        print(layer)
        weight1 = list(model_b.parameters())[layer]
        weight2 = list(model_d.parameters())[layer]
        cosine_similarity = F.cosine_similarity(weight1, weight2)
        cosine_loss += cosine_similarity
        print(f'cosine_similarity: {cosine_similarity}')
    return cosine_loss

#

### Type: 3
#### lambda_ortho = 0.01
# 
``` 
        layer_names = [2,4]
        ortho_loss = calculate_cosine_similarity_loss(model_d, model_b, layer_names)
        loss = loss_b_update.mean() + loss_d_update.mean() - (lambda_ortho / 2) * ortho_loss
``` 
- Results: 72.80%  

In [ ]:
def calculate_cosine_similarity_loss(model_b, model_d, layer_names):
    p11 = list(model_b.parameters())
    print(f'p11: {p11[2]}')
    p22 = list(model_d.parameters())
    cos_sim = 0.0
    for i in layer_names:
            p111 = p11[i].view(p11[i].size(0), -1)
            p222 = p22[i].view(p22[i].size(0), -1)
            sim = F.cosine_similarity(p111, p222)
            cos_sim += sim.mean(0)
            print(f'cos_sim: {cos_sim}')

    return cos_sim

#

### Type: 4
#### lambda_ortho = 0.01
# 
``` 
        layer_names = ['feature.2.weight', 'feature.4.weight']
        ortho_loss = calculate_cosine_similarity_loss(model_d, model_b, layer_names)
        loss = loss_b_update.mean() + loss_d_update.mean() - (lambda_ortho / 2) * ortho_loss
``` 
- Results: 77.97%  

In [ ]:
def calculate_cosine_similarity_loss(model_b, model_d, layer_names):
    cosine_loss = 0.0
    for name1, param1 in model_b.named_parameters():
        for layer_name in layer_names:
            if layer_name in name1:
                name2 = name1.replace(layer_name, layer_name.replace('features', 'classifier'))
                param2 = model_d.state_dict()[name2]
                if param1 is not None and param2 is not None and 'weight' in name1:
                    weight1 = param1.view(param1.size(0), -1)
                    weight2 = param2.view(param2.size(0), -1)
                    cosine_similarity = F.cosine_similarity(weight1, weight2)
                    cosine_loss += 1 - cosine_similarity
                    print(f'cosine_loss for {layer_name}: {cosine_loss}')
    return cosine_loss

#